Answer all questions and submit them either as an IPython notebook, LaTeX document, or Markdown document. Each question is worth 25 points.

This homework is due Friday, September 22, 2017.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
from sklearn.linear_model import LinearRegression

## Question 1

The data below provides counts of a flour beetle (Tribolium confusum) population at various points in time:

In [ ]:
days = 0,8,28,41,63,79,97,117,135,154
beetles = 2,47,192,256,768,896,1120,896,1184,1024

plt.plot(days, beetles)

An elementary model for population growth is the logistic model:

$$\frac{dN}{dt} = rN\left(1 - \frac{N}{K}\right)$$

where $N$ is population size, $t$ is time, $r$ is a growth rate parameter, and $K$ is a parameter that represents the population carrying capacity of the environment. The solution to this differential equation is given by: 

$$N_t = f(t) = \frac{KN_0}{N_0 + (K - N_0)\exp(-rt)}$$

where $N_t$ denotes the population size at time $t$. 

1. Fit the logistic growth model to the flour beetle data using optimization to minimize the sum of squared errors between model predictions and observed counts.

2. In many population modeling applications, an assumption of lognormality is adopted. The simplest assumption would be that the $\log(N_t)$ are independent and normally distributed with mean $\log[f(t)]$ and variance $\sigma^2$. Find the MLEs under this assumption, and provide estimates of standard errors and correlation between them.

Question 1.1
We can rewrite the logistic model to:

$$\frac{d(log\frac{p}{1-p})}{dt} = r, \text{where } p = \frac{N}{K}$$

So we can $logistic(p)=rt+logistic(p_0), p = \frac{N}{K} \text{and } p_0 = \frac{N_0}{K}$. Finally the solution comes as:

$$N_t = f(t) = \frac{KN_0}{N_0 + (K - N_0)\exp(-rt)}$$

Fit the logistic growth model to the flour beetle data using optimization to minimize the sum of squared errors between model predictions and observed counts.

The cost function $J$ can be expressed as:
$$J = \frac{1}{2m} (\hat{N_t} - N_t)^2 \sim (\hat{N_t} - N_t)^2$$


In [ ]:
# code for question 1.1
def costfunc(theta):
    K = theta[0]
    r = theta[1]
    N_0 = 2
    t = days
    N_t = beetles
    N_t_hat = 1.0*N_0*K/(N_0+(K-N_0)*np.exp(-r*t))
    return np.sum((N_t_hat - N_t)**2)

days = np.array(days)
beetles = np.array(beetles)
# optimization
from scipy.optimize import minimize
# intialized guess

theta = [500, 0.5]
res = minimize(costfunc, theta, method='Nelder-Mead', tol = 1e-6)
K = res.x[0]
r = res.x[1]
print("The first model using least square error gives K = %.3f, r = %.3f" % (K, r))
N_0 = 2
model1 = 1.0*N_0*K/(N_0+(K-N_0)*np.exp(-r*days))

# plot vs the original
plt.plot(days, beetles, c = 'b')
plt.plot(days, model1, c = 'r')

Question 1.2 The assumptions:

$$ log(N_t) \sim N(f(t), \sigma^2)$$
Get the log-likelihood for each data point based on the normal model and add them up to minimize. Since we need to provide estimates of standard errors and correlation between them, let's bootstrap the 10-point dataset for 1000 times and obtain $K_i$ and $r_i$ in each loop. Finally, calculate each strandard error and their correlation.

In [ ]:
# code for question 1.2
from scipy.stats import norm

def loglikelihood(theta):
    K = theta[0]
    r = theta[1]
    var = theta[2]
    t = new_days
    x = new_beetles
    N_0 = 2
    # will use minimize, multiply by -1 instead
    return -1.0*np.sum(norm.logpdf(np.log(x),loc=np.log(1.0*N_0*K/(N_0+(K-N_0)*np.exp(-r*t))),scale=var))

# intialize at each loop
n = 1000
K_para = np.zeros(n)
r_para = np.zeros(n)
var_para = np.zeros(n)
new_days = np.zeros(days.shape)
new_beetles = np.zeros(beetles.shape)

for i in range(n):
    index = np.random.choice(len(days), len(days))
    new_days = np.take(days, index)
    new_beetles = np.take(beetles, index)
    theta = [500, 0.5, np.var(np.log(new_beetles))]
    res = minimize(loglikelihood, theta, method='Nelder-Mead', tol = 1e-6)
    K_para[i] = res.x[0]
    r_para[i] = res.x[1]
    var_para[i] = res.x[2]

print("The standard error of K is %.3f." % np.var(K_para))
print("The standard error of r is %.3f." % np.var(r_para))
print("The standard error of var is %.3f." % np.var(var_para))
print("The correlation between K and r is %.3f." % np.corrcoef(K_para, r_para)[0,1])
print("The correlation between r and var is %.3f." % np.corrcoef(r_para, var_para)[0,1])
print("The correlation between var and K is %.3f." % np.corrcoef(var_para, K_para)[0,1])

## Question 2

A. Implement simulated annealing for minimizing the AIC for the baseball salary regression problem. Model your algorithm on the example given in class. 
1. Compare the effects of different cooling schedules (different temperatures and different durations at each temperature).  
2. Compare the effect of a proposal distribution that is discrete uniform over 2-neighborhoods versus one that is discrete uniform over 3-neighborhoods.

In [ ]:
baseball = pd.read_table('../data/baseball.dat', sep='\s+')
predictors = baseball.copy()
logsalary = predictors.pop('salary').apply(np.log)
nrows, ncols = predictors.shape

aic = lambda g, X, y: len(y) * np.log(sum((g.predict(X) - y)**2)/len(y)) + 2*g.rank_

def annealingfunc(cooling_schedule, neighbors= 1, color = 'r'):
    aic_values = []
    solution_current = solution_best = np.random.binomial(1, 0.5, ncols).astype(bool)
    solution_vars = predictors[predictors.columns[solution_current]]
    g = LinearRegression().fit(X=solution_vars, y=logsalary)
    aic_best = aic(g, solution_vars, logsalary)
    aic_values.append(aic_best)
    for tau in cooling_schedule:

        # Random change # 0f neighborhood
        flips = np.random.choice(ncols, neighbors, replace = False)
        for flip in flips:
            solution_current[flip] = not solution_current[flip] 
        solution_vars = predictors[predictors.columns[solution_current]]
        g = LinearRegression().fit(X=solution_vars, y=logsalary)
        aic_step = aic(g, solution_vars, logsalary)
        alpha = min(1, np.exp((aic_values[-1] - aic_step)/tau))

        if ((aic_step < aic_values[-1]) or (np.random.uniform() < alpha)):
            # Accept proposed solution
            aic_values.append(aic_step)
            if aic_step < aic_best:
                # Replace previous best with this one
                aic_best = aic_step
                solution_best = solution_current.copy()
        else:
            # Revert solution
            solution_current[flip] = not solution_current[flip]
            aic_values.append(aic_values[-1])
    # for comparison, add color argument
    plt.plot(aic_values, color)
    plt.xlim(0, len(aic_values))
    plt.xlabel('Iteration')
    plt.ylabel('AIC')
    print('Best AIC: {0}\nBest solution: {1}\nDiscovered at iteration {2}'.format(aic_best, 
                np.where(solution_best==True),
                np.where(aic_values==aic_best)[0][0]))
    plt.plot(np.where(aic_values==aic_best)[0][0], aic_best, color + 'o')

In [ ]:
# Compare the effects of different cooling schedules (different temperatures and different durations at each temperature).
tau_start = 10
# in class cases
cooling_schedule = [tau_start]*60 + [tau_start/2]*120 + [tau_start/10]*180
annealingfunc(cooling_schedule, color = 'b')
# flip the sequence
annealingfunc(cooling_schedule[::-1], color = 'g')
# try equal length
cooling_schedule = [tau_start]*120 + [tau_start/2]*120 + [tau_start/10]*120
annealingfunc(cooling_schedule, color = 'r')

Flipping the cooling schedule make the AIC values increase in last few iterations. So starting with high-value temperature is crucial. Probably choose smaller values as the iteration goes on. I prefer the original cooling schedule in our example case (blue curve).

In [ ]:
# try different ratio factors 0.9, 0.5, 0.1
cooling_schedule = [tau_start*0.9**i for i in range(360)]
annealingfunc(cooling_schedule, color = 'c')
cooling_schedule = [tau_start*0.5**i for i in range(360)]
annealingfunc(cooling_schedule, color = 'y')
cooling_schedule = [tau_start*0.1**i for i in range(360)]
annealingfunc(cooling_schedule, color = 'k')

It seems that we obtain the best and minimum AIC value for 0.9 and 0.5 ratio factor earlier than 0.1. I pefer use 0.9 or 0.8 because it makes the cooling schedule more smoothly than 0.5 and 0.1.

In [ ]:
# Compare the effect of a proposal distribution that is discrete uniform 
# over 2-neighborhoods versus one that is discrete uniform over 3-neighborhoods.
cooling_schedule = [tau_start*i/360 for i in range(360, 0, -1)]
annealingfunc(cooling_schedule, neighbors =2, color = 'b')
annealingfunc(cooling_schedule, neighbors =3, color = 'r')

Choosing 3-neighborhoods seems better than 2-neighborhoods.

B. Implement a genetic algorithm for minimizing the AIC for the baseball salary regression problem. Model your algorithm on Example 3.5. 
1. Compare the effects of using different mutation rates.  
2. Compare the effects of using different generation sizes.  
3. Instead of the selection mechanism used in the class example, try using independent selection of both parents with probabilities proportional to their fitness.

In [ ]:
def calculate_fitness(aic_values):
    P = len(aic_values)
    aic_rank = (-aic_values).argsort().argsort()+1.
    return 2.*aic_rank/(P*(P+1.))

def geneticfunc(mutation_rate, pop_size, iterations, double_selection = False, color = 'b'):
    aic_best = []
    best_solution = []
    aic_history = []
    # Initialize genotype
    current_gen = np.random.binomial(1, 0.5, pop_size*ncols).reshape((pop_size, ncols))
    for i in range(iterations):

        # Get phenotype
        current_phe = [predictors[predictors.columns[g.astype(bool)]] for g in current_gen]
        # Calculate AIC
        current_aic = np.array([aic(LinearRegression().fit(X=x, y=logsalary), x, logsalary) for x in current_phe])
        # Get lowest AIC
        aic_best.append(current_aic[np.argmin(current_aic)])
        best_solution.append(current_gen[np.argmin(current_aic)])

        # Calculate fitness according to AIC rank
        fitness = calculate_fitness(current_aic)

        # Choose first parents according to fitness
        moms = np.random.choice(range(pop_size), size=int(pop_size/2), p=fitness)
        if not double_selection:
            # Choose second parents randomly
            # in the example
            dads = np.random.choice(range(pop_size), size=int(pop_size/2))
        else:
            # Choose second parents according to fitness
            dads = np.random.choice(range(pop_size), size=int(pop_size/2), p=fitness)

        next_gen = []
        for x,y in zip(current_gen[moms], current_gen[dads]):
            # Crossover
            cross = np.random.randint(0, ncols)
            child1 = np.r_[x[:cross], y[cross:]]
            child2 = np.r_[y[:cross], x[cross:]]
            # Mutate
            m1 = np.random.binomial(1, mutation_rate, size=ncols).astype(bool)
            child1[m1] = abs(child1[m1]-1)
            m2 = np.random.binomial(1, mutation_rate, size=ncols)
            child2[m2] = abs(child1[m2]-1)
            next_gen += [child1, child2]

        # Increment generation
        current_gen = np.array(next_gen)
        # Store AIC values
        aic_history.append(current_aic)
    
    plt.plot(aic_best, color)
    plt.xlim(0, iterations)
    plt.xlabel('Iteration')
    plt.ylabel('AIC')
    print("The best and minimum AIC is " +str(min(aic_best)))

In [ ]:
# Compare the effects of using different mutation rates.
mutation_rates = [0.2, 0.1, 0.05, 0.02, 0.005]
colors = ['b', 'g', 'r', 'c', 'y']
for i in range(len(colors)):
    geneticfunc(mutation_rates[i], pop_size = 20, iterations = 200, color = colors[i])

It seems that 0.02 is better than other mutation_rates.

In [ ]:
# Compare the effects of using different generation sizes.
generation_sizes = [10, 20, 30, 40, 50]
colors = ['b', 'g', 'r', 'c', 'y']
for i in range(len(colors)):
    geneticfunc(mutation_rate = 0.02, pop_size = 20, iterations = 200, color = colors[i])

As the generation size grows (for example, 50), we obtained the best and minimum AIC.

In [ ]:
# Instead of the selection mechanism used in the class example, 
# try using independent selection of both parents with probabilities proportional to their fitness.
geneticfunc(mutation_rate = 0.02, pop_size = 20, iterations = 200, double_selection = False, color = 'b')
geneticfunc(mutation_rate = 0.02, pop_size = 20, iterations = 200, double_selection = True, color = 'r')

Use 0.02 as mutation rate, 20 as generation size. The blue curve represents our original case in the example while we choose both parents according to the fitness and show the AIC tendency in the red curve. Without comparing the AIC values (very close), the blue curve seems more gradually decreasing to the minimum while the red curve is fast dropping down. Maybe I will prefer the blue curve.

## Question 3

Use the combinatorial optimization method of your choice to obtain a solution to the traveling salesman problem for the Brazilian cities described in the lecture notes, using minimum total distance as the criterion. Use the the first city listed in the dataset as "home" (*i.e.* the trip must start and end there). I will award 5 bonus points to the best solution!

In [23]:
def parse_latlon(x):
    d, m, s = map(float, x.split(':'))
    ms = m/60. + s/3600.
    if d<0:
        return d - ms
    return d + ms

cities =  pd.read_csv('../data/brasil_capitals.txt', \
                      names=['city','lat','lon'])[['lat','lon']].applymap(parse_latlon)
cities = np.array(cities)

def distance(route):
    '''
    route is a non-replace index of cities.
    its sequence represents the traveling salesman route.
    return: distance, route
    '''
    loop = None 
    loop = np.append(route, route[0])
    res = 0
    for i in range(len(loop)-1):
        res += np.sqrt((cities[loop[i+1]][0] - cities[loop[i]][0])**2 + (cities[loop[i+1]][1] - cities[loop[i]][1])**2)
    return res

def annealingfunc(route, cooling_schedule, iteration_schedule, neighbors = 3):
    cur_distance = distance(route)
    # the simulating annealing run
    for cooling_step in range(len(cooling_schedule)):
        for iteration in range(iteration_schedule[cooling_step]):
            # random change # of index
            new_route = route.copy()
            # index in the original route
            flip_index = np.random.choice(len(route), size = neighbors, replace = False)
            # index in the new route, use the permutation of flip_index
            new_index = np.random.permutation(flip_index)
            for i, j in zip(new_index, flip_index):
                new_route[i] = route[j]
            new_distance = distance(new_route)
            # find alpha
            alpha = min(1, np.exp((cur_distance-new_distance)/cooling_schedule[cooling_step]) )
            # Accept or Reject?
            if (new_distance < cur_distance) or (np.random.uniform() < alpha):
                route = new_route
                cur_distance = new_distance
#     print("Distance:{0: .3f}, Route:{1}".format(cur_distance, route))
    return cur_distance, route 

# try different hyperparameters
tau_start_list = [10, 20, 30]
steps_list = [30, 50, 100]
factor_list = [0.9, 0.8, 0.7]
neighbors_list = [2,4,6]
distance_list = []
route_list = []

for tau_start in tau_start_list:
    for steps in steps_list:
        for factor in factor_list:
            for neighbors in neighbors_list:
                # initialize
                    cooling_schedule = [tau_start*factor**i for i in range(steps)]
                    iteration_schedule = [10]*int(steps/5) + [20]*int(steps/5) + [50]*int(steps/5) \
                    + [100]*int(steps/5) + [300]*int(steps/5)
                    route = np.random.permutation(range(len(cities)))
                    temp_distance = None
                    temp_route = None
                    temp_distance, temp_route = annealingfunc(route, cooling_schedule, iteration_schedule, neighbors = neighbors)   
                    distance_list.append(temp_distance)
                    route_list.append(temp_route)

# get the index of the smallest value in running results
ind = distance_list.index(min(distance_list))
print(distance_list[ind])
print(route_list[ind])

C:\Users\Rui\AppData\Local\conda\conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in exp


133.474010118
[10 12 15 20 17 16 13 18 21 22 23 25 24 19 14 11  9  8  7  5  6  3  2  1  0
  4]


In [24]:
# This chunk is used for saving best-so-far results.
# format, put 0 first
best_route = [ 6 , 5,  7,  8,  9, 11, 14, 19, 18, 21, 22, 23, 24, 25, 20, 17, 16, 13, 15, 12, 10,  4,  0,  1,  2, 3]
zero_index = best_route.index(0)
best_route = best_route[zero_index:] + best_route[:zero_index]
best_distance = distance(best_route)
print("%.3f"%best_distance)
print(best_route)

127.285
[0, 1, 2, 3, 6, 5, 7, 8, 9, 11, 14, 19, 18, 21, 22, 23, 24, 25, 20, 17, 16, 13, 15, 12, 10, 4]


I suppose the genetic algorithm should have better results than the stimulating annealing method. There are a lot of papers discussing it. Here I found a paper: Genetic algorithms for the traveling salesman problem by Jean-Yves Potvin, Annals of Operations Research. There are a lot of crossover operators when passing the gene from parents to the childs. I am not quite sure how to define the fitness in this question. I would like to see the code using the genetic algorithm after grading. 

## Question 4

The `../data/ebola` folder contains summarized reports of Ebola cases from three countries during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

From these data files, use `pandas` to import them and create a single data frame that includes the daily totals of new cases and deaths for each country.

In [ ]:
import os
import glob
import pandas as pd

# three paths
path1 =r'/Users/ruiwang/source/bios_8366/data/ebola/guinea_data'
path2 =r'/Users/ruiwang/source/bios_8366/data/ebola/liberia_data'
path3 =r'/Users/ruiwang/source/bios_8366/data/ebola/sl_data'

# add up
allFiles = []
for path in [path1, path2, path3]:
    allFiles += glob.glob(path + "/*.csv")

ebola = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)

ebola = pd.concat(list_)

In [ ]:
# rearrange the columns
col = ebola.columns.tolist()
for x in ['Date', 'Description', 'Totals']:
    col.remove(x)
col = ['Date', 'Description', 'Totals'] + col
ebola = ebola[col].fillna(0)
ebola = ebola[(ebola['Description'] == 'New cases of confirmed') & (ebola['Description'] == 'New deaths registered today')]